# Leçon SQL 12: Ordre d'exécution d'une requête

Maintenant que nous avons une idée de toutes les parties d'une requête, nous pouvons maintenant parler de la façon dont elles s'articulent toutes dans le contexte d'une requête complète.

Requête SELECT complète
``` mysql 
SELECT DISTINCT column, AGG_FUNC(column_or_expression), …
FROM mytable
    JOIN another_table
      ON mytable.column = another_table.column
    WHERE constraint_expression
    GROUP BY column
    HAVING constraint_expression
    ORDER BY column ASC/DESC
    LIMIT count OFFSET COUNT;
```

Chaque requête commence par rechercher les données dont nous avons besoin dans une base de données, puis filtrer ces données en quelque chose qui peut être traité et compris le plus rapidement possible. Étant donné que chaque partie de la requête est exécutée de manière séquentielle, il est important de comprendre l'ordre d'exécution afin de savoir quels résultats sont accessibles où.

## Ordre d'exécution des requêtes

### 1. FROM et JOIN

La clause FROM et les JOIN suivants sont d'abord exécutés pour déterminer l'ensemble de travail total de données interrogées. Cela inclut les sous-requêtes dans cette clause et peut entraîner la création de tables temporaires sous le capot contenant toutes les colonnes et lignes des tables jointes.

### 2. WHERE

Une fois que nous avons le jeu de données de travail total, les contraintes WHERE de premier passage sont appliquées aux lignes individuelles, et les lignes qui ne satisfont pas à la contrainte sont ignorées. Chacune des contraintes peut uniquement accéder aux colonnes directement à partir des tables demandées dans la clause FROM. Les alias de la partie SELECT de la requête ne sont pas accessibles dans la plupart des bases de données car ils peuvent inclure des expressions dépendant de parties de la requête qui n'ont pas encore été exécutées.

### 3. GROUP BY

Les lignes restantes après l'application des contraintes WHERE sont ensuite regroupées en fonction des valeurs communes de la colonne spécifiée dans la clause GROUP BY. En raison du regroupement, il n'y aura que le nombre de lignes qu'il y aura de valeurs uniques dans cette colonne. Implicitement, cela signifie que vous ne devriez avoir besoin de l'utiliser que lorsque vous avez des fonctions d'agrégation dans votre requête.

### 4. HAVING

Si la requête a une clause GROUP BY, les contraintes de la clause HAVING sont alors appliquées aux lignes groupées, supprimez les lignes groupées qui ne satisfont pas la contrainte. Comme la clause WHERE, les alias ne sont pas non plus accessibles à partir de cette étape dans la plupart des bases de données.

### 5. SELECT

Toutes les expressions de la partie SELECT de la requête sont finalement calculées.

### 6. DISTINCT

Parmi les lignes restantes, les lignes avec des valeurs en double dans la colonne marquée DISTINCT seront supprimées.

### 7. ORDER BY

Si un ordre est spécifié par la clause ORDER BY, les lignes sont ensuite triées en fonction des données spécifiées dans l'ordre croissant ou décroissant. Étant donné que toutes les expressions de la partie SELECT de la requête ont été calculées, vous pouvez référencer des alias dans cette clause.

### 8. LIMIT / OFFSET

Enfin, les lignes qui tombent en dehors de la plage spécifiée par LIMIT et OFFSET sont ignorées, laissant l'ensemble final de lignes à renvoyer à partir de la requête.

### Conclusion

Toutes les requêtes ne doivent pas nécessairement contenir toutes les parties que nous avons répertoriées ci-dessus, mais une partie de la raison pour laquelle SQL est si flexible est qu'il permet aux développeurs et aux analystes de données de manipuler rapidement les données sans avoir à écrire du code supplémentaire, le tout en utilisant simplement les clauses ci-dessus.

## Exercice

Ici se termine nos leçons sur les requêtes **SELECT**, félicitations pour l'avoir fait jusqu'ici! Cet exercice essaiera de tester votre compréhension des requêtes, alors ne vous découragez pas si vous les trouvez difficiles. Juste faites de votre mieux.

**IMPORTANT : Afin de répondre aux questions de l'exercise d'insérer la requête entre les doubles quotes " "**

In [2]:
import pandas as pd

def to_df(s):
    l= [elt.split(';') for elt in s.replace("	", ";").split('\n')]
    return pd.DataFrame(l[1:], columns=l[0])

In [3]:
from pandasql import sqldf

In [4]:
x="""Id	Title	Director	Year	Length_minutes
1	Toy Story	John Lasseter	1995	81
2	A Bug's Life	John Lasseter	1998	95
3	Toy Story 2	John Lasseter	1999	93
4	Monsters, Inc.	Pete Docter	2001	92
5	Finding Nemo	Andrew Stanton	2003	107
6	The Incredibles	Brad Bird	2004	116
7	Cars	John Lasseter	2006	117
8	Ratatouille	Brad Bird	2007	115
9	WALL-E	Andrew Stanton	2008	104
10	Up	Pete Docter	2009	101
11	Toy Story 3	Lee Unkrich	2010	103
12	Cars 2	John Lasseter	2011	120
13	Brave	Brenda Chapman	2012	102
14	Monsters University	Dan Scanlon	2013	110"""
Movies = to_df(x)

In [5]:
Movies

,Id,Title,Director,Year,Length_minutes
0,1,Toy Story,John Lasseter,1995,81
1,2,A Bug's Life,John Lasseter,1998,95
2,3,Toy Story 2,John Lasseter,1999,93
3,4,"Monsters, Inc.",Pete Docter,2001,92
4,5,Finding Nemo,Andrew Stanton,2003,107
5,6,The Incredibles,Brad Bird,2004,116
6,7,Cars,John Lasseter,2006,117
7,8,Ratatouille,Brad Bird,2007,115
8,9,WALL-E,Andrew Stanton,2008,104
9,10,Up,Pete Docter,2009,101


In [7]:
y="""Movie_id	Rating	Domestic_sales	International_sales
5	8.2	380843261	555900000
14	7.4	268492764	475066843
8	8	206445654	417277164
12	6.4	191452396	368400000
3	7.9	245852179	239163000
6	8	261441092	370001000
9	8.5	223808164	297503696
11	8.4	415004880	648167031
1	8.3	191796233	170162503
7	7.2	244082982	217900167
10	8.3	293004164	438338580
4	8.1	289916256	272900000
2	7.2	162798565	200600000
13	7.2	237283207	301700000"""

In [8]:
BoxOffice = to_df(y)

In [9]:
BoxOffice

,Movie_id,Rating,Domestic_sales,International_sales
0,5,8.2,380843261,555900000
1,14,7.4,268492764,475066843
2,8,8,206445654,417277164
3,12,6.4,191452396,368400000
4,3,7.9,245852179,239163000
5,6,8,261441092,370001000
6,9,8.5,223808164,297503696
7,11,8.4,415004880,648167031
8,1,8.3,191796233,170162503
9,7,7.2,244082982,217900167


1. Trouvez le nombre de films que chaque réalisateur a réalisé


In [16]:
QUERY1 = ""

In [17]:
sqldf(QUERY1)

,Id,Title,Director,Year,Length_minutes
0,1,Toy Story,John Lasseter,1995,81
1,2,A Bug's Life,John Lasseter,1998,95
2,3,Toy Story 2,John Lasseter,1999,93
3,4,"Monsters, Inc.",Pete Docter,2001,92
4,5,Finding Nemo,Andrew Stanton,2003,107
5,6,The Incredibles,Brad Bird,2004,116
6,7,Cars,John Lasseter,2006,117
7,8,Ratatouille,Brad Bird,2007,115
8,9,WALL-E,Andrew Stanton,2008,104
9,10,Up,Pete Docter,2009,101


2. Trouvez le total des ventes nationales et internationales pouvant être attribuées à chaque directeur

In [16]:
QUERY2 = ""

In [17]:
sqldf(QUERY2)

,Id,Title,Director,Year,Length_minutes
0,1,Toy Story,John Lasseter,1995,81
1,2,A Bug's Life,John Lasseter,1998,95
2,3,Toy Story 2,John Lasseter,1999,93
3,4,"Monsters, Inc.",Pete Docter,2001,92
4,5,Finding Nemo,Andrew Stanton,2003,107
5,6,The Incredibles,Brad Bird,2004,116
6,7,Cars,John Lasseter,2006,117
7,8,Ratatouille,Brad Bird,2007,115
8,9,WALL-E,Andrew Stanton,2008,104
9,10,Up,Pete Docter,2009,101
